In [2]:
import ast
import re

# --------- Phase 1: Syntax Checking ----------
def check_syntax(code):
    try:
        ast.parse(code)
        print("✔ No syntax or indentation errors.")
    except SyntaxError as e:
        print(f"❌ Syntax error: {e}")

# --------- Phase 2: Unreachable Code Detection ----------
class UnreachableCodeVisitor(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        has_return = False
        for stmt in node.body:
            if has_return:
                print(f"⚠ Unreachable code detected at line {stmt.lineno}")
            if isinstance(stmt, ast.Return):
                has_return = True
        self.generic_visit(node)

def detect_unreachable_code(code):
    tree = ast.parse(code)
    UnreachableCodeVisitor().visit(tree)

# --------- Phase 3: Static Analysis ----------
def static_analysis(code):
    if "eval(" in code or "exec(" in code:
        print("⚠ Use of eval/exec detected. This can be insecure.")
    if "import os" in code:
        print("🔎 Note: os module usage found. Ensure sandboxing if needed.")

# --------- Phase 4: Three Address Code (TAC) Generation ----------
temp_counter = 0
def new_temp():
    global temp_counter
    temp = f"t{temp_counter}"
    temp_counter += 1
    return temp

def generate_tac(code):
    tree = ast.parse(code)
    tac = []

    class TACVisitor(ast.NodeVisitor):
        def visit_Assign(self, node):
            target = node.targets[0].id
            if isinstance(node.value, ast.BinOp):
                left = self.visit(node.value.left)
                right = self.visit(node.value.right)
                op = self.op_to_str(node.value.op)
                temp = new_temp()
                tac.append(f"{temp} = {left} {op} {right}")
                tac.append(f"{target} = {temp}")
            else:
                value = self.visit(node.value)
                tac.append(f"{target} = {value}")

        def visit_Constant(self, node):
            return str(node.value)

        def visit_Name(self, node):
            return node.id

        def visit_BinOp(self, node):
            left = self.visit(node.left)
            right = self.visit(node.right)
            op = self.op_to_str(node.op)
            temp = new_temp()
            tac.append(f"{temp} = {left} {op} {right}")
            return temp

        def visit_Expr(self, node):
            if isinstance(node.value, ast.Call):
                if isinstance(node.value.func, ast.Name) and node.value.func.id == "print":
                    args = [self.visit(arg) for arg in node.value.args]
                    for arg in args:
                        tac.append(f"print {arg}")

        def visit_FunctionDef(self, node):
            tac.append(f"\nfunction {node.name} {{")
            for stmt in node.body:
                self.visit(stmt)
            tac.append("}\n")

        def visit_Return(self, node):
            val = self.visit(node.value)
            tac.append(f"return {val}")

        def op_to_str(self, op):
            return {
                ast.Add: "+", ast.Sub: "-", ast.Mult: "*", ast.Div: "/"
            }.get(type(op), "?")

    TACVisitor().visit(tree)
    return "\n".join(tac)

# --------- Phase 5: Python to C Translator ----------
def translate_python_to_c(code):
    lines = code.strip().split('\n')
    translated = []
    indent_level = 0
    indent = lambda: "    " * indent_level
    declared_vars = {}

    def infer_type(value):
        if re.match(r"^\d+\.\d+$", value):
            return "float"
        elif re.match(r"^\d+$", value):
            return "int"
        else:
            return "char*" if '"' in value or "'" in value else "int"

    for line in lines:
        stripped = line.strip()
        if not stripped:
            continue

        if stripped.startswith("def "):
            func_name = stripped.split()[1].split("(")[0]
            translated.append(f"{indent()}void {func_name}() {{")
            indent_level += 1

        elif stripped.startswith("return "):
            val = stripped.split("return", 1)[1].strip()
            translated.append(f"{indent()}return {val};")

        elif stripped.startswith("print("):
            content = stripped[len("print("):-1].strip()
            if content.isidentifier():
                var_type = declared_vars.get(content, "int")
                fmt = "%d" if var_type == "int" else "%f"
                translated.append(f'{indent()}printf("{fmt}\\n", {content});')
            else:
                translated.append(f'{indent()}printf("{content}\\n");')

        elif "=" in stripped and not stripped.startswith("for") and not stripped.startswith("while"):
            var, val = map(str.strip, stripped.split("=", 1))
            if var not in declared_vars:
                var_type = infer_type(val)
                declared_vars[var] = var_type
                translated.append(f"{indent()}{var_type} {var} = {val};")
            else:
                translated.append(f"{indent()}{var} = {val};")

        elif stripped.startswith("for "):
            match = re.match(r"for (\w+) in range\((.*)\):", stripped)
            if match:
                var, range_vals = match.groups()
                parts = list(map(str.strip, range_vals.split(',')))
                if len(parts) == 1:
                    start, end, step = "0", parts[0], "1"
                elif len(parts) == 2:
                    start, end = parts
                    step = "1"
                else:
                    start, end, step = parts
                translated.append(f"{indent()}for (int {var} = {start}; {var} < {end}; {var} += {step}) {{")
                indent_level += 1

        elif stripped.startswith("while "):
            condition = stripped[6:-1] if stripped.endswith(":") else stripped[6:]
            translated.append(f"{indent()}while ({condition}) {{")
            indent_level += 1

        elif stripped == "pass":
            translated.append(f"{indent()}// pass")

        elif not line.startswith(" ") and indent_level > 0:
            while indent_level > 0:
                indent_level -= 1
                translated.append(f"{indent()}}}")

        else:
            translated.append(f"{indent()}// {stripped}  // Not yet supported")

    while indent_level > 0:
        indent_level -= 1
        translated.append(f"{indent()}}}")

    return "\n".join(translated)

# --------- Phase 6: C Code Optimizer ----------
def optimize_c_code(code):
    return "\n".join([line for line in code.split('\n') if line.strip() != ";"])

# --------- 🧪 Sample Input Code ----------
sample_code = """
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

# Example usage:
num = int(input("Enter a number: "))
if is_prime(num):
    print(f"{num} is a prime number.")
else:
    print(f"{num} is not a prime number.")

"""

# --------- 🚀 Compiler Pipeline Execution ----------
print("🔹 Static Analysis 🔹")
check_syntax(sample_code)
detect_unreachable_code(sample_code)
static_analysis(sample_code)

print("\n🔹 Three Address Code 🔹")
tac_output = generate_tac(sample_code)
print(tac_output)

print("\n🔹 Python to C Translation 🔹")
c_code = translate_python_to_c(sample_code)
print(c_code)

print("\n🔹 Optimized C Code 🔹")
print(optimize_c_code(c_code))


🔹 Static Analysis 🔹
✔ No syntax or indentation errors.

🔹 Three Address Code 🔹

function is_prime {
return False
t0 = n ? 0.5
t1 = None + 1
t2 = n ? i
return False
return True
}

num = None
print None
print None

🔹 Python to C Translation 🔹
void is_prime() {
    int if n < = 1:;
    return False;
    for (int i = 2; i < int(n**0.5) + 1; i += 1) {
        int if n % i = = 0:;
        return False;
        return True;
    }
}
char* num = int(input("Enter a number: "));
// if is_prime(num):  // Not yet supported
printf("f"{num} is a prime number."\n");
// else:  // Not yet supported
printf("f"{num} is not a prime number."\n");

🔹 Optimized C Code 🔹
void is_prime() {
    int if n < = 1:;
    return False;
    for (int i = 2; i < int(n**0.5) + 1; i += 1) {
        int if n % i = = 0:;
        return False;
        return True;
    }
}
char* num = int(input("Enter a number: "));
// if is_prime(num):  // Not yet supported
printf("f"{num} is a prime number."\n");
// else:  // Not yet supporte